<a href="https://colab.research.google.com/github/K-SAHASRA/GNN-modelling/blob/main/gnn_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install pandas
!pip install scikit-learn
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu117.html





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.9 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 67.4 MB/s eta 0:00:00


In [2]:

!pip uninstall torch-scatter -y
!pip uninstall torch-geometric -y
!pip uninstall torch-sparse -y
!pip install torch==2.0.0+cu117 torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter torch-sparse torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cu117.html


Found existing installation: torch-scatter 2.1.2+pt20cu117
Uninstalling torch-scatter-2.1.2+pt20cu117:
  Successfully uninstalled torch-scatter-2.1.2+pt20cu117
Found existing installation: torch-geometric 2.6.1
Uninstalling torch-geometric-2.6.1:
  Successfully uninstalled torch-geometric-2.6.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 860.8 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 105.2 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━

In [3]:
from google.colab import files

uploaded = files.upload()


Saving 100-iterations data combined.csv to 100-iterations data combined.csv


In [4]:
from google.colab import files

uploaded = files.upload()


Saving All Exp-data-as-joinbyfield-2024-07-03 22_02_41.csv to All Exp-data-as-joinbyfield-2024-07-03 22_02_41.csv


In [5]:
import torch
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from torch_scatter import scatter_mean


train_file_path = '/content/100-iterations data combined.csv'
train_data = pd.read_csv(train_file_path, low_memory=False)

test_file_path = '/content/All Exp-data-as-joinbyfield-2024-07-03 22_02_41.csv'
test_data = pd.read_csv(test_file_path, low_memory=False)
print("Original test data columns:", test_data.columns)

def preprocess_data(data):
    data = data.drop(columns=['sep=', 'Unnamed: 1'], errors='ignore')

    if 'Time' in data.columns:
        data['Time'] = data['Time'].astype(str)
        data_cleaned = data.drop(columns=['Time'])
    else:
        data_cleaned = data.copy()

    def custom_convert(series):
        series = series.str.replace(' K', '000')
        series = series.str.replace(' M', '000000')
        return pd.to_numeric(series, errors='coerce')

    if 'Number of Users' in data_cleaned.columns:
        data_cleaned['Number of Users'] = pd.to_numeric(data['Number of Users'], errors='coerce')
        data_cleaned = data_cleaned[data_cleaned['Number of Users'] != 0]

    columns_to_remove = [
        "Median Response Time",
        "Number of Users",
        "Current Response Time Percentile 95",
        "Current Response Time Percentile 50"
    ]

    data_cleaned = data_cleaned.drop(columns=[col for col in columns_to_remove if col in data_cleaned.columns], errors='ignore')

    data_cleaned = data_cleaned.apply(lambda col: custom_convert(col) if col.dtype == 'object' else col)
    data_cleaned = data_cleaned.fillna(0)

    if 'Average Response Time' in data_cleaned.columns:
        data_cleaned = data_cleaned[data_cleaned['Average Response Time'] != 0]

    return data_cleaned

train_data_cleaned = preprocess_data(train_data)
test_data_cleaned = preprocess_data(test_data)
print("Train columns:", train_data_cleaned.columns)
print("Test columns:", test_data_cleaned.columns)


Original test data columns: Index(['Time', 'Avg Host Node1 CPU Usage', 'Host Node1 CPU 0 Usage',
       'Host Node1 CPU 1 Usage', 'Avg Host Node2 CPU Usage',
       'Host Node2 CPU 0 Usage', 'Host Node2 CPU 1 Usage',
       'Avg Host Node3 CPU Usage', 'Host Node3 CPU 0 Usage',
       'Host Node3 CPU 1 Usage', 'Avg Host Node4 CPU Usage',
       'Host Node4 CPU 0 Usage', 'Host Node4 CPU 1 Usage',
       'checkoutservice CPU Usage', 'currencyservice CPU Usage',
       'emailservice CPU Usage', 'frontend 1 CPU Usage',
       'frontend 2 CPU Usage', 'loadgenerator CPU Usage',
       'paymentservice CPU Usage', 'productcatalogservice CPU Usage',
       'recommendationservice 1 CPU Usage',
       'recommendationservice 2 CPU Usage', 'redis-cart CPU Usage',
       'shippingservice CPU Usage', 'Average Response Time',
       'Median Response Time', 'Current Response Time Percentile 95',
       'Current Response Time Percentile 50', 'Number of Users',
       'Host Node 1 Mem Usage', 'Host Node 2

In [17]:
#run this cell at the end
test_data_cleaned = preprocess_data(test_data)


ground_truth = test_data_cleaned['Average Response Time'].values[time_window:]

ground_truth_df = pd.DataFrame({
    'Time Step': range(time_window + 1, time_window + 1 + len(ground_truth)),
    'Actual Average Response Time': ground_truth
})

ground_truth_file_path = '/content/ground_truth_values.csv'
ground_truth_df.to_csv(ground_truth_file_path, index=False)

files.download(ground_truth_file_path)
#run this at the end

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:



common_columns = train_data_cleaned.columns.intersection(test_data_cleaned.columns)
train_data_cleaned = train_data_cleaned[common_columns]
test_data_cleaned = test_data_cleaned[common_columns]

scaler = MinMaxScaler()
normalized_train_data = scaler.fit_transform(train_data_cleaned)
normalized_test_data = scaler.transform(test_data_cleaned)

target_scaler = MinMaxScaler()
target_column_train = train_data_cleaned[['Average Response Time']]
target_scaler.fit(target_column_train)


normalized_train_data = torch.tensor(normalized_train_data, dtype=torch.float32)
normalized_test_data = torch.tensor(normalized_test_data, dtype=torch.float32)

def sliding_window_view(data, window_size):
    windows = []
    for i in range(len(data) - window_size):
        windows.append(data[i:i + window_size])
    return torch.stack(windows)


time_window = 10
train_node_features_time_series = sliding_window_view(normalized_train_data, time_window)
test_node_features_time_series = sliding_window_view(normalized_test_data, time_window)

average_response_time_column = 24
train_target_time_series = torch.tensor(normalized_train_data[time_window:, average_response_time_column], dtype=torch.float32).view(-1, 1)

num_nodes = train_node_features_time_series.size(1)
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

batch_size = train_node_features_time_series.size(0)

batch_index = torch.arange(batch_size).repeat_interleave(num_nodes)

train_graph_data = Data(x=train_node_features_time_series.view(-1, train_node_features_time_series.size(2)),
                        edge_index=edge_index, y=train_target_time_series, batch=batch_index)

test_graph_data = Data(x=test_node_features_time_series.view(-1, test_node_features_time_series.size(2)),
                       edge_index=edge_index, batch=batch_index)







<ipython-input-7-227284b4e681>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_target_time_series = torch.tensor(normalized_train_data[time_window:, average_response_time_column], dtype=torch.float32).view(-1, 1)


In [11]:
# # class GNNModel(nn.Module):
# #     def __init__(self, in_channels, hidden_channels, out_channels):
# #         super(GNNModel, self).__init__()
# #         self.conv1 = GCNConv(in_channels, hidden_channels)
# #         self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
# #         self.fc1 = nn.Linear(hidden_channels * 4, hidden_channels)
# #         self.fc2 = nn.Linear(hidden_channels, out_channels)
# #         self.output_activation = nn.ReLU()

# #     def forward(self, data):
# #         x, edge_index = data.x, data.edge_index
# #         x = self.conv1(x, edge_index)
# #         x = self.output_activation(x)
# #         x = self.att1(x, edge_index)
# #         x = torch.relu(x)
# #         x = self.fc1(x)
# #         x = torch.relu(x)
# #         x = self.fc2(x)
# #         x = self.output_activation(x)

# #         # Removed scatter_mean to avoid issues with batch indices
# #         return x.mean(dim=0)  # Simple average over all nodes


# # # Model instantiation
# # model = GNNModel(in_channels=train_node_features_time_series.size(2), hidden_channels=32, out_channels=1)
# # optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# # criterion = nn.MSELoss()

# # # Training the model
# # def train(model, data, epochs=100):
# #     model.train()
# #     for epoch in range(epochs):
# #         optimizer.zero_grad()
# #         out = model(data)
# #         loss = criterion(out, data.y)
# #         loss.backward()
# #         optimizer.step()
# #         print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# # # Train the model
# # train(model, train_graph_data)

# # # Ensure shapes match expected sizes
# # print(f"Training Data Shape: {train_graph_data.x.shape}")
# # print(f"Test Data Shape: {test_graph_data.x.shape}")
# class GNNModel(nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super(GNNModel, self).__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
#         self.fc1 = nn.Linear(hidden_channels * 4, hidden_channels)
#         self.fc2 = nn.Linear(hidden_channels, out_channels)
#         self.output_activation = nn.ReLU()

#     def forward(self, data):
#         x, edge_index = data.x, data.edge_index
#         x = self.conv1(x, edge_index)
#         x = self.output_activation(x)
#         x = self.att1(x, edge_index)
#         x = torch.relu(x)
#         x = self.fc1(x)
#         x = torch.relu(x)
#         x = self.fc2(x)
#         return x


# def train(model, data, epochs=100):
#     model.train()
#     for epoch in range(epochs):
#         optimizer.zero_grad()
#         out = model(data)
#         loss = criterion(out, data.y)

#         loss.backward()
#         optimizer.step()
#         print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# train(model, train_graph_data)



import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv
from torch.optim import Adam

class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        self.fc1 = nn.Linear(hidden_channels * 4, hidden_channels)  # Adjusted for GAT heads
        self.fc2 = nn.Linear(hidden_channels, out_channels)
        self.output_activation = nn.ReLU()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.output_activation(x)
        x = self.att1(x, edge_index)
        x = torch.relu(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)


        return x.mean(dim=0, keepdim=True)





in_channels = train_node_features_time_series.size(2)
hidden_channels = 32
out_channels = 1

model = GNNModel(in_channels=in_channels, hidden_channels=hidden_channels, out_channels=out_channels)

optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

def train(model, data, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)


        target = data.y

        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, train_graph_data)



Epoch 1, Loss: 0.017321744933724403
Epoch 2, Loss: 0.015201113186776638
Epoch 3, Loss: 0.014651356264948845
Epoch 4, Loss: 0.015072831884026527
Epoch 5, Loss: 0.01544332504272461
Epoch 6, Loss: 0.015392975881695747
Epoch 7, Loss: 0.015113440342247486
Epoch 8, Loss: 0.014830796048045158
Epoch 9, Loss: 0.014671538956463337
Epoch 10, Loss: 0.014658111147582531
Epoch 11, Loss: 0.014740603044629097
Epoch 12, Loss: 0.014843727461993694
Epoch 13, Loss: 0.014908540062606335
Epoch 14, Loss: 0.014911392703652382
Epoch 15, Loss: 0.014860687777400017
Epoch 16, Loss: 0.014782964251935482
Epoch 17, Loss: 0.01470849011093378
Epoch 18, Loss: 0.014660424552857876
Epoch 19, Loss: 0.014648549258708954
Epoch 20, Loss: 0.014667773619294167
Epoch 21, Loss: 0.014701878651976585
Epoch 22, Loss: 0.014731569215655327
Epoch 23, Loss: 0.014743110164999962
Epoch 24, Loss: 0.014733204618096352
Epoch 25, Loss: 0.01470821350812912
Epoch 26, Loss: 0.01467945333570242
Epoch 27, Loss: 0.014657530002295971
Epoch 28, Loss

In [12]:

test_graphs = []
for i in range(test_node_features_time_series.size(0)):
    test_graph = Data(x=test_node_features_time_series[i].view(-1, test_node_features_time_series.size(2)),
                      edge_index=edge_index)
    test_graphs.append(test_graph)



In [15]:


def predict_individual_graphs(model, test_graphs):
    model.eval()
    predictions = []

    with torch.no_grad():
        for graph in test_graphs:
            pred = model(graph)
            predictions.append(pred)

    return torch.stack(predictions, dim=0)

#had a error regarding 2d array format

predicted_response_times = predict_individual_graphs(model, test_graphs)


predicted_response_times_2d = predicted_response_times.view(-1, 1).numpy()
predicted_response_times_original_scale = target_scaler.inverse_transform(predicted_response_times_2d)


predicted_response_times_original_scale = predicted_response_times_original_scale.reshape(predicted_response_times.shape)

for i, pred in enumerate(predicted_response_times_original_scale, start=time_window + 1):
    print(f"Time step {i}: Predicted Average Response Time = {pred.item():.4f}")


Streaming output truncated to the last 5000 lines.
Time step 927: Predicted Average Response Time = 29.2431
Time step 928: Predicted Average Response Time = 29.2516
Time step 929: Predicted Average Response Time = 29.6117
Time step 930: Predicted Average Response Time = 29.9968
Time step 931: Predicted Average Response Time = 29.9515
Time step 932: Predicted Average Response Time = 29.9170
Time step 933: Predicted Average Response Time = 29.6846
Time step 934: Predicted Average Response Time = 29.4196
Time step 935: Predicted Average Response Time = 29.1331
Time step 936: Predicted Average Response Time = 28.9598
Time step 937: Predicted Average Response Time = 28.7925
Time step 938: Predicted Average Response Time = 29.3582
Time step 939: Predicted Average Response Time = 30.0435
Time step 940: Predicted Average Response Time = 30.4121
Time step 941: Predicted Average Response Time = 30.5789
Time step 942: Predicted Average Response Time = 30.8638
Time step 943: Predicted Average Resp

In [16]:

predicted_values = predicted_response_times_original_scale

time_window = 10
time_steps = list(range(time_window + 1, time_window + 1 + len(predicted_values)))

predicted_values_flat = [pred[0] for pred in predicted_values]

predictions_df = pd.DataFrame({
    'Time Step': time_steps,
    'Predicted Average Response Time': predicted_values_flat
})

output_file_path = '/content/predicted_response_times_with_timestamps.csv'
predictions_df.to_csv(output_file_path, index=False)

files.download(output_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>